#Functions

In [1]:
import imageio
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import imageio
import cv2
import pandas as pd
import random

In [2]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

def filter(intersections, max_count=100):
  # A list holds the SSE values for each k

  if len(intersections) == 0:
    return 0, []

  kmeans_kwargs = {
    "init": "random",
    "n_init": 10,
    "max_iter": 300,
    "random_state": 42,
  }

  sse = []
  cluster_centers = []
  for k in range(0, min(len(intersections), max_count)):
    kmeans = KMeans(n_clusters=k+1, **kmeans_kwargs)
    kmeans.fit(intersections)
    sse.append(kmeans.inertia_)
    cluster_centers.append(kmeans.cluster_centers_)

  k = np.sum(sse>=(np.mean(sse)-0.25*np.std(sse)))

  return k, cluster_centers[k-1]

In [3]:
def predict_combined(image_id, threshold, size, model):
  intersections = []

  gif_black = imageio.mimread(f"/content/P03_1000_test/test_black/black_{image_id}.gif")
  gif_map = imageio.mimread(f"/content/P03_1000_test/test_map/map_{image_id}.gif")

  gif_black = np.flip(cv2.cvtColor(gif_black[0], cv2.COLOR_RGB2BGR), axis=0) #flip y axis because original y origin is below
  gif_map = np.flip(cv2.cvtColor(gif_map[0], cv2.COLOR_RGB2BGR), axis=0) #flip y axis because original y origin is below
  cords = [(x, y) for y in range(size, 1000-size, 5) for x in range(size,1000-size,5)]

  grid_black = np.zeros((len(cords),size,size,3), int)
  grid_map = np.zeros((len(cords),size,size,3), int)

  index = 0

  for x, y in cords:

    subimage_black = gif_black[y-size//2:y+size//2+1, x-size//2:x+size//2+1]
    subimage_map = gif_map[y-size//2:y+size//2+1, x-size//2:x+size//2+1]

    if subimage_black.shape[0] == size and subimage_black.shape[1] == size:
      grid_black[index] = subimage_black #np.concatenate((grid, [subimage]))
      grid_map[index] = subimage_map
      index += 1

  prediction_map = model.predict([grid_map, grid_black])

  for (pred_0, pred_1), (x, y) in zip(prediction_map, cords):
    if pred_1>=threshold:
        intersections.append([x, y])

  return intersections

In [4]:
def predict(image_id, threshold, size, model):
  intersections = []

  gif_black = imageio.mimread(f"/content/P03_1000_test/test_black/black_{image_id}.gif")
  gif_black = np.flip(cv2.cvtColor(gif_black[0], cv2.COLOR_RGB2BGR), axis=0) #flip y axis because original y origin is below


  cords = [(x, y) for y in range(size, 1000-size, 5) for x in range(size,1000-size,5)]

  grid_black = np.zeros((len(cords),size,size,3), int)
  index = 0

  for x, y in cords:

    subimage_black = gif_black[y-size//2:y+size//2+1, x-size//2:x+size//2+1]

    if subimage_black.shape[0] == size and subimage_black.shape[1] == size:
      grid_black[index] = subimage_black #np.concatenate((grid, [subimage]))
      index += 1

  prediction_map = model.predict(grid_black)

  for (pred_0, pred_1), (x, y) in zip(prediction_map, cords):
    if pred_1>=threshold: #predict accept thrshold
        intersections.append([x, y])

  return intersections

#Test predict


In [5]:
from tensorflow.keras.models import load_model

combined_model = load_model('/content/combined_model.keras')
size = 63

In [13]:
df_test_2 = pd.read_csv('/content/P03_1000_test/test.csv', header=None)
df_test_2.columns = ['image_id']
df_test_2['image_id'] = df_test_2['image_id'].str.replace('.gif', '')
df_test_2

<ipython-input-13-e138e77a2540>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test_2['image_id'] = df_test_2['image_id'].str.replace('.gif', '')


,image_id
0,605000_6478000
1,549000_6579000
2,639000_6432000
3,625000_6400000
4,409000_6448000
...,...
995,594000_6499000
996,575000_6459000
997,421000_6528000
998,412000_6536000


In [ ]:
import gc

threshold = 0.999
index = 0

with open('test_out_999_combined_2.csv', 'w') as file:
  for index2, row in df_test_2.iterrows():
    image_id = row['image_id']
    print(index, image_id)
    intersections = predict_combined(image_id, threshold, size, combined_model)
    gc.collect()
    k, clusters = filter(intersections)

    gc.collect()
    file.write(f'{image_id}, ')
    for (x, y) in clusters:
      file.write(f'{int(x)}-{int(y)}, ')
    file.write(f'\n')

    index += 1


0 625000_6400000
958/958 [==============================] - 3s 4ms/step
1 409000_6448000
958/958 [==============================] - 3s 4ms/step
2 526000_6410000
958/958 [==============================] - 3s 4ms/step
3 659000_6525000
958/958 [==============================] - 3s 4ms/step
4 527000_6480000
958/958 [==============================] - 3s 4ms/step
5 457000_6531000
958/958 [==============================] - 3s 4ms/step
6 688000_6468000
958/958 [==============================] - 3s 4ms/step
7 657000_6571000
958/958 [==============================] - 3s 4ms/step
8 524000_6482000
958/958 [==============================] - 3s 4ms/step
9 495000_6460000
958/958 [==============================] - 3s 4ms/step
10 709000_6593000
958/958 [==============================] - 3s 4ms/step
11 652000_6532000
958/958 [==============================] - 3s 4ms/step
12 438000_6482000
958/958 [==============================] - 3s 4ms/step
13 485000_6573000
958/958 [==============================] - 